In [1]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')


# Perth basin.


In [2]:
import os

# Importing GemPy
import gempy as gp

# Importing auxiliary libraries
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda"

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\ASUS\AppData\Local\conda\conda\envs\tf\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
data_path = 'https://raw.githubusercontent.com/cgre-aachen/gempy_data/master/'
geo_model = gp.create_model('Perth_Basin')

In [4]:
gp.init_data(geo_model,
             extent=[337000, 400000, 6640000, 6710000, -18000, 1000],
             resolution=[100, 100, 100],
             path_i=data_path + "/data/input_data/Perth_basin/Paper_GU2F_sc_faults_topo_Points.csv",
             path_o=data_path + "/data/input_data/Perth_basin/Paper_GU2F_sc_faults_topo_Foliations.csv")

Active grids: ['regular']


Perth_Basin  2021-10-10 16:44

In [5]:
geo_model.surfaces

,surface,series,order_surfaces,color,id
0,Lesueur,Default series,1,#015482,1
1,Permian,Default series,2,#9f0052,2
2,Woodada_Kockatea,Default series,3,#ffbe00,3
3,Yarragadee,Default series,4,#728f02,4
4,Eneabba,Default series,5,#443988,5
5,Cattamarra,Default series,6,#ff3f20,6
6,Cadda,Default series,7,#5DA629,7
7,Cretaceous,Default series,8,#4878d0,8
8,Darling,Default series,9,#ee854a,9
9,Urella_North,Default series,10,#6acc64,10


In [6]:
del_surfaces = ['Cadda' ,'Woodada_Kockatea', 'Cattamarra','Abrolhos_Transfer',
                             'Hypo_fault_W',
                           'Hypo_fault_E',
                           'Coomallo',
                           'Eneabba_South',
                            'Urella_North',
                           'Urella_South',
                            'Darling']

In [7]:
geo_model.delete_surfaces(del_surfaces, remove_data=True) #'Eneabba_South','Coomallo'#'Hypo_fault_E','Hypo_fault_W',#
#'Darling','Urella_North','Urella_South'

,surface,series,order_surfaces,color,id
0,Lesueur,Default series,1,#015482,1
1,Permian,Default series,2,#9f0052,2
3,Yarragadee,Default series,3,#728f02,3
4,Eneabba,Default series,4,#443988,4
7,Cretaceous,Default series,5,#4878d0,5
16,basement,Basement,1,#82c6e2,6


%debug



In [8]:
geo_model.stack

,order_series,BottomRelation,isActive,isFault,isFinite
Default series,1,Erosion,True,False,False
Basement,2,Erosion,False,False,False


In [9]:
gp.map_stack_to_surfaces(geo_model,
                          {
                         "Sedimentary_Series": ['Cretaceous',
                                                  'Yarragadee',
                                                  'Eneabba',
                                                  'Lesueur',
                                                  'Permian','Cadda', 'Woodada_Kockatea', 'Cattamarra']
                           })

,surface,series,order_surfaces,color,id
0,Lesueur,Sedimentary_Series,1,#015482,1
1,Permian,Sedimentary_Series,2,#9f0052,2
3,Yarragadee,Sedimentary_Series,3,#728f02,3
4,Eneabba,Sedimentary_Series,4,#443988,4
7,Cretaceous,Sedimentary_Series,5,#4878d0,5
16,basement,Basement,1,#82c6e2,6


In [10]:
geo_model.series

,order_series,BottomRelation,isActive,isFault,isFinite
Sedimentary_Series,1,Erosion,True,False,False
Basement,2,Erosion,False,False,False


In [11]:
order_series = ["Sedimentary_Series", 'Basement' ]

geo_model.reorder_series(order_series)

,order_series,BottomRelation,isActive,isFault,isFinite
Sedimentary_Series,1,Erosion,True,False,False
Basement,2,Erosion,False,False,False


Drop input data from the deleted series:




In [12]:
geo_model.surface_points.df.dropna(inplace=True)
geo_model.orientations.df.dropna(inplace=True)

## Select which series are faults




In [ ]:
geo_model.faults

In [ ]:
geo_model.set_is_fault([ "fault_Abrolhos_Transfer" ])

### Fault Network




In [ ]:
geo_model.faults.faults_relations_df

In [ ]:
fr = geo_model.faults.faults_relations_df.values

In [ ]:
fr[:,:-2] = False
fr

In [ ]:
geo_model.set_fault_relation(fr)

%matplotlib inline



In [ ]:
gp.plot_2d(geo_model, direction=['z'])

In [ ]:
geo_model.set_topography(source='random')

In [13]:
gp.plot_3d(geo_model)

In [14]:
interp_data = gp.set_interpolator(geo_model,
                                  compile_theano=True,
                                  theano_optimizer='fast_run', gradient=False,
                                  dtype='float32')

Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_run
Device:  cpu
Precision:  float32
Number of faults:  0
Compilation Done!
Kriging values: 
                       values
range                96072.9
$C_o$            2.19762e+08
drift equations       [3, 3]


In [15]:
gp.compute_model(geo_model)

C:\Users\ASUS\AppData\Local\conda\conda\envs\tf\lib\site-packages\theano\tensor\slinalg.py:255: LinAlgWarning: Ill-conditioned matrix (rcond=1.1665e-10): result may not be accurate.
  rval = scipy.linalg.solve(A, b)



Lithology ids 
  [6. 6. 6. ... 5. 5. 5.] 

In [ ]:

gp.plot_2d(geo_model, cell_number=[25])

In [ ]:
gp.plot_2d(geo_model, cell_number=[25], series_n=-1, show_scalar=True)

In [ ]:
gp.plot_2d(geo_model, cell_number=[12], direction=["y"], show_data=True, show_topography=True)

sphinx_gallery_thumbnail_number = 6



In [ ]:
gp.plot_3d(geo_model, show_topography=True)

### Times

#### Fast run

-  1M voxels:

   -  CPU: intel® Core™ i7-7700HQ CPU @ 2.80GHz × 8 15 s ± 1.02 s per
      loop (mean ± std. dev. of 7 runs, 1 loop each)
   -  GPU (4gb) not enough memmory
   -  Ceres 1M voxels 2080 851 ms

-  250k voxels

   -  GPU 1050Ti: 3.11 s ± 11.8 ms per loop (mean ± std. dev. of 7 runs,
      1 loop each)
   -  CPU: intel® Core™ i7-7700HQ CPU @ 2.80GHz × 8 2.27 s ± 47.3 ms
   -  

#### Fast Compile

-  250k voxels

   -  GPU 1050Ti: 3.7 s ± 11.8 ms per loop (mean ± std. dev. of 7 runs,
      1 loop each)
   -  CPU: intel® Core™ i7-7700HQ CPU @ 2.80GHz × 8 14.2 s ± 51.1 ms per
      loop (mean ± std. dev. of 7 runs, 1 loop each)




%%timeit
gp.compute_model(geo_model)



ver = np.load('ver.npy')
sim = np.load('sim.npy')
lith_block = np.load('lith.npy')



In [ ]:
gp.save_model(geo_model)

In [ ]:
import pickle

In [ ]:
an_obj = Model_4


file_to_store = open("stored_object_3.pickle", "wb")
pickle.dump(an_obj, file_to_store)


file_to_store.close()